## After talking with Riley, he suggested that I shift my focus from time-series-analysis, and more towards topic modeling ##

So, for my sources I'll do the following:
- Reddit: try to collect the actual submissions or at LEAST the title of the submissions. Maybe comments (secondary)
- arXiv: Abstract/ summary of the paper (not the whole paper)
- Job posts: The limited crappy findings
- News API: The headline and whatever text News API supports
- Google: Nothin doing
- Wiki: just do a TM of the page. Or better yet do TM over time of the page. We'll see

### First, arXiv, because I think it'll be the easiest data to get ###

In [3]:
import pandas as pd
import re
import matplotlib.pyplot as plt
import urllib
import time
import feedparser
%matplotlib inline

In [7]:
def arx_and_recreation(search_term, start_idx, scope='ti'):
    '''uses urllib, time, and feedparser'''
    #first escape search terms
    search_term= search_term.replace('"',"%22").replace(" ", "+");
    # set wait time and iteration step
    iterstep= 200;
    wait_time= 2 #make this 3 to 'play nice' with the api. I am nice I promise! but impatient atm
    base_url = 'http://export.arxiv.org/api/query?'
    start= start_idx
    date_dict={
        "date":[],
        "article_id": [],
        "summary":[],
        "source": "arXiv"
    }
    while True:
        response= urllib.request.urlopen(base_url+f"search_query={scope}:{search_term}&sortBy=submittedDate&sortOrder=ascending&start={start}&max_results={iterstep}")
        feed= feedparser.parse(response)
        if not feed.entries:
            print('query complete')
            print(f"There should be {feed.feed.opensearch_totalresults} results?")
            break
        date_dict['date'].extend([entry.published for entry in feed.entries])
        date_dict['article_id'].extend([entry.id.split('/abs/')[-1] for entry in feed.entries])
        date_dict['summary'].extend([entry.summary.replace("\n", " ") for entry in feed.entries])
        print(f"gathering results {start} to {start + iterstep-1} ")
        start = start + iterstep
        time.sleep(wait_time)
        
    return pd.DataFrame(date_dict)
        
    

In [16]:
#collect all cat:cs.AI from arXiv
# ai1 = arx_and_recreation("cs.AI", 0, scope='cat')

# ai2= arx_and_recreation("cs.AI", 10000, scope='cat')

# ai3 = arx_and_recreation("cs.AI", 12200, scope='cat')

# ai4 = arx_and_recreation("cs.AI", 14400, scope='cat')

gathering results 0 to 199 
gathering results 200 to 399 
gathering results 400 to 599 
gathering results 600 to 799 
gathering results 800 to 999 
gathering results 1000 to 1199 
gathering results 1200 to 1399 
gathering results 1400 to 1599 
gathering results 1600 to 1799 
gathering results 1800 to 1999 
gathering results 2000 to 2199 
gathering results 2200 to 2399 
gathering results 2400 to 2599 
gathering results 2600 to 2799 
gathering results 2800 to 2999 
gathering results 3000 to 3199 
gathering results 3200 to 3399 
gathering results 3400 to 3599 
gathering results 3600 to 3799 
gathering results 3800 to 3999 
gathering results 4000 to 4199 
gathering results 4200 to 4399 
gathering results 4400 to 4599 
gathering results 4600 to 4799 
gathering results 4800 to 4999 
gathering results 5000 to 5199 
gathering results 5200 to 5399 
gathering results 5400 to 5599 
gathering results 5600 to 5799 
gathering results 5800 to 5999 
gathering results 6000 to 6199 
gathering results 62

In [23]:
# all_ai = pd.concat([ai1, ai2, ai3, ai4], ignore_index=True)

# all_ai.head()

# all_ai['date']= pd.DatetimeIndex(all_ai['date']).normalize()

# all_ai.to_csv('./new_hope_data/arxiv_csAI.csv', index=False)

### It took re-running my function 4 times to get all the articles.###

In [2]:
ai = pd.read_csv('./new_hope_data/arxiv_csAI.csv')

In [3]:
corpus = ai['summary']

In [37]:
corpus = list(corpus)

In [39]:
# Going to modify this code to use my corpus instead of the 20newsgroups
# Author: Olivier Grisel <olivier.grisel@ensta.org>
#         Lars Buitinck
#         Chyi-Kwei Yau <chyikwei.yau@gmail.com>
# License: BSD 3 clause

from __future__ import print_function
from time import time

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.datasets import fetch_20newsgroups

n_samples = len(corpus)
n_features = 1000
n_components = 10
n_top_words = 20


def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()


# Load the 20 newsgroups dataset and vectorize it. We use a few heuristics
# to filter out useless terms early on: the posts are stripped of headers,
# footers and quoted replies, and common English words, words occurring in
# only one document or in at least 95% of the documents are removed.

# print("Loading dataset...")
t0 = time()
# dataset = fetch_20newsgroups(shuffle=True, random_state=1,
#                              remove=('headers', 'footers', 'quotes'))
data_samples = corpus[:n_samples]
# print("done in %0.3fs." % (time() - t0))

# Use tf-idf features for NMF.
print("Extracting tf-idf features for NMF...")
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2,
                                   max_features=n_features,
                                   stop_words='english')
t0 = time()
tfidf = tfidf_vectorizer.fit_transform(data_samples)
print("done in %0.3fs." % (time() - t0))

# Use tf (raw term count) features for LDA.
print("Extracting tf features for LDA...")
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2,
                                max_features=n_features,
                                stop_words='english')
t0 = time()
tf = tf_vectorizer.fit_transform(data_samples)
print("done in %0.3fs." % (time() - t0))
print()

# Fit the NMF model
print("Fitting the NMF model (Frobenius norm) with tf-idf features, "
      "n_samples=%d and n_features=%d..."
      % (n_samples, n_features))
t0 = time()
nmf = NMF(n_components=n_components, random_state=1,
          alpha=.1, l1_ratio=.5).fit(tfidf)
print("done in %0.3fs." % (time() - t0))

print("\nTopics in NMF model (Frobenius norm):")
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(nmf, tfidf_feature_names, n_top_words)

# Fit the NMF model
print("Fitting the NMF model (generalized Kullback-Leibler divergence) with "
      "tf-idf features, n_samples=%d and n_features=%d..."
      % (n_samples, n_features))
t0 = time()
nmf = NMF(n_components=n_components, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
          l1_ratio=.5).fit(tfidf)
print("done in %0.3fs." % (time() - t0))

print("\nTopics in NMF model (generalized Kullback-Leibler divergence):")
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(nmf, tfidf_feature_names, n_top_words)

print("Fitting LDA models with tf features, "
      "n_samples=%d and n_features=%d..."
      % (n_samples, n_features))
lda = LatentDirichletAllocation(n_components=n_components, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)
t0 = time()
lda.fit(tf)
print("done in %0.3fs." % (time() - t0))

print("\nTopics in LDA model:")
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)

Extracting tf-idf features for NMF...
done in 1.331s.
Extracting tf features for LDA...
done in 1.289s.

Fitting the NMF model (Frobenius norm) with tf-idf features, n_samples=16008 and n_features=1000...
done in 2.311s.

Topics in NMF model (Frobenius norm):
Topic #0: image features method classification based task images dataset language methods text approach visual datasets using feature art proposed object results
Topic #1: logic programs semantics programming answer reasoning program set asp fuzzy rules logics language propositional theory order stable sets argumentation logical
Topic #2: algorithm problem search problems algorithms optimization optimal time solution solutions planning constraint constraints number solving results local based heuristic new
Topic #3: learning reinforcement policy deep rl tasks learn policies machine reward environment task function action methods exploration algorithms state agent value
Topic #4: network neural networks deep training layer architec

In [8]:
# # Gonna try the same thing with arXiv's stat.ML category
# ml1 = arx_and_recreation("stat.ML", 0, scope='cat')

# ml2 = arx_and_recreation("stat.ML", 800, scope='cat')

# ml3 = arx_and_recreation("stat.ML", 1400, scope='cat')

# ml4 = arx_and_recreation("stat.ML", 10400, scope='cat')

# ml5 = arx_and_recreation("stat.ML", 17000, scope='cat')

# ml6 = arx_and_recreation("stat.ML", 18400, scope='cat')

gathering results 0 to 199 
gathering results 200 to 399 
gathering results 400 to 599 
gathering results 600 to 799 
query complete
There should be 18961 results?


In [16]:
# all_ml = pd.concat([ml1, ml2, ml3, ml4, ml5, ml6], ignore_index=True)

# all_ml['date']= pd.DatetimeIndex(all_ml['date']).normalize()

# all_ml.to_csv('./new_hope_data/arxiv_csML.csv', index=False)

In [4]:
all_ml = pd.read_csv('./new_hope_data/arxiv_csML.csv')

In [5]:
ml_corpus = list(all_ml['summary'])

In [4]:
# Going to modify this code to use my corpus instead of the 20newsgroups
# Author: Olivier Grisel <olivier.grisel@ensta.org>
#         Lars Buitinck
#         Chyi-Kwei Yau <chyikwei.yau@gmail.com>
# License: BSD 3 clause

from __future__ import print_function
from time import time

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.datasets import fetch_20newsgroups

n_samples = len(ml_corpus)
n_features = 1000
n_components = 10
n_top_words = 20


def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()


# Load the 20 newsgroups dataset and vectorize it. We use a few heuristics
# to filter out useless terms early on: the posts are stripped of headers,
# footers and quoted replies, and common English words, words occurring in
# only one document or in at least 95% of the documents are removed.

# print("Loading dataset...")
t0 = time()
# dataset = fetch_20newsgroups(shuffle=True, random_state=1,
#                              remove=('headers', 'footers', 'quotes'))
data_samples = ml_corpus[:n_samples]
# print("done in %0.3fs." % (time() - t0))

# Use tf-idf features for NMF.
print("Extracting tf-idf features for NMF...")
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2,
                                   max_features=n_features,
                                   stop_words='english')
t0 = time()
tfidf = tfidf_vectorizer.fit_transform(data_samples)
print("done in %0.3fs." % (time() - t0))

# Use tf (raw term count) features for LDA.
print("Extracting tf features for LDA...")
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2,
                                max_features=n_features,
                                stop_words='english')
t0 = time()
tf = tf_vectorizer.fit_transform(data_samples)
print("done in %0.3fs." % (time() - t0))
print()

# Fit the NMF model
print("Fitting the NMF model (Frobenius norm) with tf-idf features, "
      "n_samples=%d and n_features=%d..."
      % (n_samples, n_features))
t0 = time()
nmf = NMF(n_components=n_components, random_state=1,
          alpha=.1, l1_ratio=.5).fit(tfidf)
print("done in %0.3fs." % (time() - t0))

print("\nTopics in NMF model (Frobenius norm):")
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(nmf, tfidf_feature_names, n_top_words)

# Fit the NMF model
print("Fitting the NMF model (generalized Kullback-Leibler divergence) with "
      "tf-idf features, n_samples=%d and n_features=%d..."
      % (n_samples, n_features))
t0 = time()
nmf = NMF(n_components=n_components, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
          l1_ratio=.5).fit(tfidf)
print("done in %0.3fs." % (time() - t0))

print("\nTopics in NMF model (generalized Kullback-Leibler divergence):")
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(nmf, tfidf_feature_names, n_top_words)

print("Fitting LDA models with tf features, "
      "n_samples=%d and n_features=%d..."
      % (n_samples, n_features))
lda = LatentDirichletAllocation(n_components=n_components, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)
t0 = time()
lda.fit(tf)
print("done in %0.3fs." % (time() - t0))

print("\nTopics in LDA model:")
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)

Extracting tf-idf features for NMF...
done in 1.676s.
Extracting tf features for LDA...
done in 1.505s.

Fitting the NMF model (Frobenius norm) with tf-idf features, n_samples=18961 and n_features=1000...
done in 3.012s.

Topics in NMF model (Frobenius norm):
Topic #0: data method features classification time based analysis approach proposed methods feature using high real information prediction model datasets dimensional set
Topic #1: learning machine tasks reinforcement deep task policy algorithms agent learn supervised framework rl state performance systems multi new transfer agents
Topic #2: algorithm optimization gradient convex stochastic convergence algorithms problem function problems rate optimal functions descent non bounds linear regret method bound
Topic #3: model models inference bayesian latent variational variables gaussian posterior distribution process likelihood probabilistic distributions markov parameters mixture graphical sampling generative
Topic #4: neural networ

In [8]:
# I'll do it the gensim way:
from gensim import corpora, models, similarities
from gensim.matutils import Sparse2Corpus
from time import time

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [11]:
n_samples = len(ml_corpus)
n_features = 1000
n_components = 10
n_top_words = 20

# Use tf (raw term count) features for LDA.
print("Extracting tf features for LDA...")
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2,
                                max_features=n_features,
                                stop_words='english')
t0 = time()
tf = tf_vectorizer.fit_transform(ml_corpus)
print("done in %0.3fs." % (time() - t0))
print()

Extracting tf features for LDA...
done in 1.660s.



In [12]:
dict((v, k) for k, v in tf_vectorizer.vocabulary_.items())

{253: 'discuss',
 36: 'algorithms',
 308: 'estimating',
 295: 'entropy',
 358: 'finite',
 811: 'sequences',
 522: 'long',
 729: 'range',
 645: 'particular',
 173: 'consider',
 305: 'estimate',
 134: 'code',
 157: 'compression',
 34: 'algorithm',
 192: 'convergence',
 810: 'sequence',
 508: 'length',
 847: 'space',
 918: 'time',
 792: 'scaling',
 714: 'proposed',
 785: 'sample',
 48: 'applied',
 899: 'systems',
 603: 'non',
 88: 'best',
 814: 'set',
 153: 'complex',
 902: 'task',
 884: 'subject',
 975: 'various',
 181: 'constraints',
 853: 'specific',
 320: 'example',
 642: 'parameters',
 276: 'effective',
 988: 'wide',
 939: 'types',
 685: 'present',
 610: 'novel',
 51: 'approach',
 698: 'problem',
 533: 'makes',
 67: 'assumptions',
 766: 'required',
 354: 'filter',
 371: 'free',
 229: 'designed',
 628: 'optimizing',
 769: 'respect',
 174: 'considered',
 553: 'measure',
 239: 'different',
 587: 'nature',
 807: 'sense',
 519: 'local',
 969: 'variance',
 626: 'optimization',
 958: 'uses'

In [14]:
# Store this dictionary for later
id2word = dict((v, k) for k, v in tf_vectorizer.vocabulary_.items())
import pickle
with open('./new_hope_data/ml_pickle.dict', 'wb') as handle:
    pickle.dump(id2word, handle)


In [15]:
testthang= corpora.Dictionary.load("./new_hope_data/ml_pickle.dict")

AttributeError: 'dict' object has no attribute '_load_specials'

In [33]:
# ml_gen_corpus = Sparse2Corpus(tf)

# # Serialize and store this corpus:
# corpora.MmCorpus.serialize('./new_hope_data/ml_corpus.mm', ml_gen_corpus)

In [13]:
corpus = corpora.MmCorpus('./new_hope_data/ml_corpus.mm')

In [39]:
#my corpus has raw term counts for each abstract. Now I want to transform to TfIDF for gensim LSI or LDA
tfidf = models.TfidfModel(corpus)

In [40]:
tfidf